In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import optuna
from sklearn import datasets, metrics
from sklearn.model_selection import KFold, StratifiedKFold,GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from statistics import mean
import matplotlib.pyplot as plt
import seaborn as sns
from lightgbm import LGBMClassifier
%matplotlib inline


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


In [2]:
train_df=pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test_df=pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
greeks_df=pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
train_1_df=pd.concat([train_df,greeks_df["Alpha"]],axis=1)
train_2_df=pd.concat([train_df,greeks_df["Beta"]],axis=1)
train_3_df=pd.concat([train_df,greeks_df["Gamma"]],axis=1)
train_4_df=pd.concat([train_df,greeks_df["Delta"]],axis=1)
train_6_df=pd.concat([train_df,greeks_df["Epsilon"]],axis=1)
train_Id=train_df["Id"]
test_ID=test_df["Id"]
train_Class=train_df["Class"]
test_df.isnull().sum()
train_df.shape

(617, 58)

In [3]:
train_df["Class"]

0      1
1      0
2      0
3      0
4      1
      ..
612    0
613    0
614    0
615    0
616    0
Name: Class, Length: 617, dtype: int64

In [4]:
train_df

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,fd3dafe738fd,0.149555,3130.05946,123.763599,9.513984,13.020852,3.499305,0.077343,8.545512,2.804172,...,0.173229,1.26092,0.067730,8.967128,217.148554,8095.932828,24.640462,69.191944,21.978000,0
613,fd895603f071,0.435846,5462.03438,85.200147,46.551007,15.973224,5.979825,0.025882,12.622906,3.777550,...,10.223150,1.24236,0.426699,35.896418,496.994214,3085.308063,29.648928,124.808872,0.145340,0
614,fd8ef6377f76,0.427300,2459.10720,130.138587,55.355778,10.005552,8.070549,0.025578,15.408390,1.229900,...,0.173229,0.49706,0.067730,19.962092,128.896894,6474.652866,26.166072,119.559420,21.978000,0
615,fe1942975e40,0.363205,1263.53524,85.200147,23.685856,8.138688,7.981959,0.025578,7.524588,1.229900,...,9.256996,0.78764,0.670527,24.594488,72.611063,1965.343176,25.116750,37.155112,0.184622,0


In [5]:
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split,GridSearchCV
from sklearn.metrics import roc_auc_score,accuracy_score,confusion_matrix,ConfusionMatrixDisplay,RocCurveDisplay

import time 
from xgboost import XGBClassifier
from sklearn.metrics import log_loss

In [6]:
train_df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [7]:
train_df.dtypes

Id        object
AB       float64
AF       float64
AH       float64
AM       float64
AR       float64
AX       float64
AY       float64
AZ       float64
BC       float64
BD       float64
BN       float64
BP       float64
BQ       float64
BR       float64
BZ       float64
CB       float64
CC       float64
CD       float64
CF       float64
CH       float64
CL       float64
CR       float64
CS       float64
CU       float64
CW       float64
DA       float64
DE       float64
DF       float64
DH       float64
DI       float64
DL       float64
DN       float64
DU       float64
DV       float64
DY       float64
EB       float64
EE       float64
EG       float64
EH       float64
EJ        object
EL       float64
EP       float64
EU       float64
FC       float64
FD       float64
FE       float64
FI       float64
FL       float64
FR       float64
FS       float64
GB       float64
GE       float64
GF       float64
GH       float64
GI       float64
GL       float64
Class      int64
dtype: object

In [8]:
test_df.dtypes

Id      object
AB     float64
AF     float64
AH     float64
AM     float64
AR     float64
AX     float64
AY     float64
AZ     float64
BC     float64
BD     float64
BN     float64
BP     float64
BQ     float64
BR     float64
BZ     float64
CB     float64
CC     float64
CD     float64
CF     float64
CH     float64
CL     float64
CR     float64
CS     float64
CU     float64
CW     float64
DA     float64
DE     float64
DF     float64
DH     float64
DI     float64
DL     float64
DN     float64
DU     float64
DV     float64
DY     float64
EB     float64
EE     float64
EG     float64
EH     float64
EJ      object
EL     float64
EP     float64
EU     float64
FC     float64
FD     float64
FE     float64
FI     float64
FL     float64
FR     float64
FS     float64
GB     float64
GE     float64
GF     float64
GH     float64
GI     float64
GL     float64
dtype: object

In [9]:
num_cols = train_df.select_dtypes(include=["float64"]).columns.tolist()
cat_cols = train_df.select_dtypes(include=["object"]).columns.tolist()

cat_cols.remove("Id")

In [10]:
total_row_train=train_df.shape
total_row_test=test_df.shape
type(total_row_train[0])

int

In [11]:
num=len(train_df[train_df["Class"]==1])
percentage=num/(total_row_train[0])*100
print(f"there are {num} Classes are '1', which is {percentage:.2f}% total training dataset.")

num=len(train_df[train_df["Class"]==0])
percentage=num/(total_row_train[0])*100
print(f"there are {num} Classes are '0', which is {percentage:.2f}% total training dataset.")


there are 108 Classes are '1', which is 17.50% total training dataset.
there are 509 Classes are '0', which is 82.50% total training dataset.


In [12]:
train_df.isnull().sum().loc[train_df.isnull().sum()>0]

BQ    60
CB     2
CC     3
DU     1
EL    60
FC     1
FL     1
FS     2
GL     1
dtype: int64

In [13]:
NULL=["NULL_EL","NULL_BQ"]

In [14]:
train_df["NULL_EL"]=0
train_df.loc[train_df["EL"]=="NULL","NULL_EL"]=1
print(train_df["NULL_EL"])

0      0
1      0
2      0
3      0
4      0
      ..
612    0
613    0
614    0
615    0
616    0
Name: NULL_EL, Length: 617, dtype: int64


In [15]:
train_df["NULL_BQ"]=0
train_df.loc[train_df["BQ"]=="NULL","NULL_BQ"]=1
print(train_df["NULL_BQ"])

0      0
1      0
2      0
3      0
4      0
      ..
612    0
613    0
614    0
615    0
616    0
Name: NULL_BQ, Length: 617, dtype: int64


In [16]:
test_df.isnull().sum().loc[train_df.isnull().sum()>0]

BQ    0
CB    0
CC    0
DU    0
EL    0
FC    0
FL    0
FS    0
GL    0
dtype: int64

In [17]:
num_cols = train_df.select_dtypes(include=["float64","int64"]).columns.tolist()
cat_cols = train_df.select_dtypes(include=["object"]).columns.tolist()
cat_cols.remove("Id")


In [18]:
train_df

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FS,GB,GE,GF,GH,GI,GL,Class,NULL_EL,NULL_BQ
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1,0,0
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0,0,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0,0,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0,0,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,fd3dafe738fd,0.149555,3130.05946,123.763599,9.513984,13.020852,3.499305,0.077343,8.545512,2.804172,...,0.067730,8.967128,217.148554,8095.932828,24.640462,69.191944,21.978000,0,0,0
613,fd895603f071,0.435846,5462.03438,85.200147,46.551007,15.973224,5.979825,0.025882,12.622906,3.777550,...,0.426699,35.896418,496.994214,3085.308063,29.648928,124.808872,0.145340,0,0,0
614,fd8ef6377f76,0.427300,2459.10720,130.138587,55.355778,10.005552,8.070549,0.025578,15.408390,1.229900,...,0.067730,19.962092,128.896894,6474.652866,26.166072,119.559420,21.978000,0,0,0
615,fe1942975e40,0.363205,1263.53524,85.200147,23.685856,8.138688,7.981959,0.025578,7.524588,1.229900,...,0.670527,24.594488,72.611063,1965.343176,25.116750,37.155112,0.184622,0,0,0


In [19]:
def competition_log_loss(y_true,y_pred):
    N_0=np.sum(1-y_true)
    N_1=np.sum(y_true)
    p_1=np.clip(y_pred,1e-15,1-(1e-15))
    p_0=1-p_1
    log_loss_0=-np.sum((1-y_true)*np.log(p_0)/N_0)
    log_loss_1=-np.sum(y_true*np.log(p_1))
    return (log_loss_0+log_loss_1)/2

def balanced_log_loss(y_true,y_pred):
    N_0=np.sum(1-y_true)
    N_1=np.sum(y_true)
    p_1=np.clip(y_pred,1e-15,1-(1e-15))
    p_0=1-p_1
    log_loss_0=-np.sum((1-y_true)*np.log(p_0)/N_0)
    log_loss_1=-np.sum(y_true*np.log(p_1))
    w_0=1/N_0
    w_1=1/N_1
    balanced_log_loss=2*(w_0*log_loss_0+w_1*log_loss_1)/(w_0+w_1)
    return balanced_log_loss/(N_0+N_1)

In [20]:
def lgb_metric(y_true,y_pred):
    return "balanced_log_loss",balanced_log_loss(y_true,y_pred),False

In [21]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder,StandardScaler
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score

                     
scaler=MinMaxScaler()
scaled_numeric_df=pd.DataFrame(scaler.fit_transform(train_df[num_cols]),columns=num_cols)

encoder=OneHotEncoder(sparse_output=False,handle_unknown="ignore")
encoded_cat_df=pd.DataFrame(encoder.fit_transform(train_df[cat_cols]),columns=encoder.get_feature_names_out(cat_cols))

processed_df=pd.concat([train_Id,scaled_numeric_df,encoded_cat_df],axis=1)
processed_df


,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,GE,GF,GH,GI,GL,Class,NULL_EL,NULL_BQ,EJ_A,EJ_B
0,000ff2bfdfe9,0.021082,0.102347,0.000000,0.030632,0.000000,0.000000,0.000000,0.180337,0.002958,...,0.000000,0.013846,0.176983,0.362261,0.005425,1.0,0.0,0.0,0.0,1.0
1,007255e47698,0.010541,0.027589,0.000000,0.053864,0.000000,0.078048,0.000000,0.284500,0.000000,...,0.000000,0.194527,0.274495,0.164135,1.000000,0.0,0.0,0.0,1.0,0.0
2,013f2bd269f5,0.063949,0.085715,0.000000,0.046519,0.000000,0.160575,0.000000,0.265013,0.000000,...,0.011229,0.095035,0.258994,0.180218,0.008910,0.0,0.0,0.0,0.0,1.0
3,043ac50845d5,0.028110,0.127285,0.019180,0.117854,0.000000,0.079462,0.000000,0.215235,0.000000,...,0.006882,0.014475,0.425143,0.470820,0.007039,0.0,0.0,0.0,0.0,1.0
4,044fb8a146ec,0.049192,0.124246,0.000000,0.017417,0.000000,0.086300,0.002841,0.000000,0.069008,...,0.051588,0.059198,0.500829,0.185841,0.004345,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,fd3dafe738fd,0.011244,0.103085,0.021132,0.010101,0.028583,0.074511,0.005030,0.144730,0.001076,...,0.101448,0.056218,0.211871,0.358883,1.000000,0.0,0.0,0.0,1.0,0.0
613,fd895603f071,0.058327,0.184921,0.000000,0.069139,0.045868,0.140533,0.000030,0.259345,0.001742,...,0.297867,0.021368,0.281649,0.651146,0.006562,0.0,0.0,0.0,0.0,1.0
614,fd8ef6377f76,0.056922,0.079539,0.024625,0.083174,0.010930,0.196180,0.000000,0.337644,0.000000,...,0.039506,0.044942,0.233126,0.623561,1.000000,0.0,0.0,0.0,1.0,0.0
615,fe1942975e40,0.046381,0.037583,0.000000,0.032691,0.000000,0.193822,0.000000,0.116032,0.000000,...,0.000000,0.013579,0.218507,0.190531,0.008349,0.0,0.0,0.0,0.0,1.0


In [22]:
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
processed_df['fold'] = -1

for fold, (train_idx, test_idx) in enumerate(skf.split(processed_df, greeks_df['Beta'])):
    processed_df.loc[test_idx, 'fold'] = fold

processed_df.groupby('fold')["Class"].value_counts()

fold  Class
0     0.0      101
      1.0       23
1     0.0       98
      1.0       26
2     0.0      101
      1.0       22
3     0.0      103
      1.0       20
4     0.0      106
      1.0       17
Name: Class, dtype: int64

In [23]:
final_valid_prediction={}
final_test_prediction=[]
scores=[]
log_losses=[]
balanced_log_losses=[]
weights=[]



for fold in range(5):
    train_processed_df = processed_df[processed_df['fold'] != fold]
    valid_processed_df = processed_df[processed_df['fold'] == fold]
    valid_ids = valid_processed_df.Id.values.tolist()

    X_train, y_train = train_processed_df.drop(['Id', 'Class', 'fold'], axis=1), train_processed_df['Class']
    X_valid, y_valid = valid_processed_df.drop(['Id', 'Class', 'fold'], axis=1), valid_processed_df['Class']
    watchlist=[(X_train,y_train),(X_valid,y_valid)]
    model=XGBClassifier(n_estimators=180,max_depth=6,colsample_bytree=0.7)
    
    try:
        model.fit(X_train,y_train,eval_set=watchlist,early_stopping_rounds=300,verbose=0)
    
        y_pred = model.predict_proba(X_valid)
        
        logloss = log_loss(y_valid, y_pred)
        balanced_logloss = balanced_log_loss(y_valid, y_pred[:, 1])
        log_losses.append(logloss)
        balanced_log_losses.append(balanced_logloss)
        weights.append(1/balanced_logloss)
    finally:
        print(y_valid)
        
    print(f"Fold: {fold}, log loss: {round(logloss, 3)}, balanced los loss: {round(balanced_logloss, 3)}")

print()
print("Log Loss")
print(log_losses)
print(np.mean(log_losses), np.std(log_losses))
print()
print("Balanced Log Loss")
print(balanced_log_losses)
print(np.mean(balanced_log_losses), np.std(balanced_log_losses))
print()
print("Weights")
print(weights)
    


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


0      1.0
3      0.0
5      0.0
6      0.0
15     0.0
      ... 
572    0.0
576    1.0
601    0.0
606    0.0
608    1.0
Name: Class, Length: 124, dtype: float64
Fold: 0, log loss: 0.211, balanced los loss: 0.174


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


4      1.0
7      0.0
16     0.0
19     0.0
28     0.0
      ... 
603    0.0
609    0.0
610    0.0
612    0.0
616    0.0
Name: Class, Length: 124, dtype: float64
Fold: 1, log loss: 0.195, balanced los loss: 0.266


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


1      0.0
9      0.0
13     1.0
20     0.0
21     0.0
      ... 
597    0.0
600    0.0
602    0.0
607    0.0
613    0.0
Name: Class, Length: 123, dtype: float64
Fold: 2, log loss: 0.182, balanced los loss: 0.254


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


2      0.0
10     1.0
11     0.0
12     0.0
17     0.0
      ... 
589    0.0
596    0.0
598    0.0
605    0.0
614    0.0
Name: Class, Length: 123, dtype: float64
Fold: 3, log loss: 0.168, balanced los loss: 0.219


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


8      0.0
14     0.0
18     0.0
22     0.0
30     0.0
      ... 
594    0.0
599    0.0
604    0.0
611    0.0
615    0.0
Name: Class, Length: 123, dtype: float64
Fold: 4, log loss: 0.136, balanced los loss: 0.142

Log Loss
[0.2107886591003413, 0.19485990370884798, 0.1817893122562763, 0.1676326497857856, 0.13622210299495624]
0.1782585255692415 0.02540177890354943

Balanced Log Loss
[0.1740864875343674, 0.2657540559577729, 0.2544629857006495, 0.2194331561257547, 0.1421314647450598]
0.21117363001272085 0.04703931909772486

Weights
[5.744271219227078, 3.762877659180093, 3.9298446382940777, 4.557196449505156, 7.035739776506862]


In [24]:
test_df=test_df.drop(["Id"],axis=1)


In [25]:
for i in NULL:
    test_df[i]=0
    
test_df

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FR,FS,GB,GE,GF,GH,GI,GL,NULL_EL,NULL_BQ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [26]:
num_cols2 = test_df.select_dtypes(include=["float64","int64"]).columns.tolist()
cat_cols2 = test_df.select_dtypes(include=["object"]).columns.tolist()

In [27]:

imputer=SimpleImputer(strategy="mean")
numeric_df2=pd.DataFrame(imputer.fit_transform(test_df[num_cols2]),columns=num_cols2
                )

scaler=MinMaxScaler()
scaled_numeric_df2=pd.DataFrame(scaler.fit_transform(numeric_df2),columns=num_cols2)

encoder=OneHotEncoder(sparse_output=False,handle_unknown="ignore")
encoder_cat_df2=pd.DataFrame(encoder.fit_transform(test_df[cat_cols2]),columns=encoder.get_feature_names_out(cat_cols2))
pred_df2=pd.concat([scaled_numeric_df2,encoder_cat_df2],axis=1)
pred_df2["EJ_B"]=0

pred_df2

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,GB,GE,GF,GH,GI,GL,NULL_EL,NULL_BQ,EJ_A,EJ_B
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [28]:
pred_df=pd.DataFrame(model.predict_proba(pred_df2),columns=["class_0","class_1"])


In [29]:
pred_df

,class_0,class_1
0,0.895895,0.104105
1,0.895895,0.104105
2,0.895895,0.104105
3,0.895895,0.104105
4,0.895895,0.104105


In [30]:
submission_df = pd.concat([test_ID,pred_df],axis=1)
submission_df.to_csv('submission.csv', index=False)

In [31]:
submission_df

,Id,class_0,class_1
0,00eed32682bb,0.895895,0.104105
1,010ebe33f668,0.895895,0.104105
2,02fa521e1838,0.895895,0.104105
3,040e15f562a2,0.895895,0.104105
4,046e85c7cc7f,0.895895,0.104105
